Το συγκεκριμένο αρχείο κώδικα αποτελεί τον πρώτο Crawler, πιο συγκεκριμένα εντοπίζει τα εξής:

1. Κοινωνικά δίκτυα στο website της επιχείρησης
2. URL κοινωνικών δικτύων στο website της επιχείρησης
3. Αν το website έχει Multi languge option
4. Αν το website έχει Search option
5. Αν το website έχει Blog
6. Αν το website έχει Mobile app
7. Αν το website έχει Eshop

In [1]:
def first_crawler(url):
    from bs4 import BeautifulSoup
    import requests
    import re
    from selenium import webdriver
    import time 
    res = []
    results = []
    results.append(url)
    url,r = check_redirects(url)
    soup = BeautifulSoup(r.content,'lxml')
    ##check if domain contains Facebook logo
    soc_nets, soc_nets_urls = social_networks(soup)
    results.append(soc_nets)
    results.append(soc_nets_urls)
    ###Multilanguage option
    mul_lang = soup.find('a',href=re.compile(r'/en|/usa|/uk|/it|/ru|lang=*|-en.html|[.]en[.]|en|_en|EN|en-UK/|eng|lang'))
    mul_lang1 = soup.find('img',src=re.compile(r'en|usa|uk|it|ru|uk|de|UK|EN|gr|GR|lang|greek|english|EnglishFlag|Flag|flag|lang|language'))
    if mul_lang == None and mul_lang1 == None:
        mullng_opt = 0
    else:
        mullng_opt = 1
    results.append(mullng_opt)
    news_letter1 = soup.find('input',{'name':re.compile(r'[Ee]?mail|EMAIL|e-mail|E-mail')})
    news_letter2 = soup.find('img',{'src':re.compile(r'[Nn]ewsletter|NEWSLETTER')})
    news_letter3 = soup.find('input',{'placeholder':re.compile(r'[Ee]?mail|EMAIL|e-mail|E-mail|E-mail...')})
    news_letter4 = soup.find('input',{'id':re.compile(r'[Ee]?mail|EMAIL|e-mail|E-mail|E-mail...|[nN]ewsletter|NEWSLETTER')})
    if news_letter1 != None or news_letter2 != None or news_letter3 !=None or news_letter4 !=None:
        newsletter = 1
    else:
        newsletter = 0
    results.append(newsletter)
    search_opt1 = soup.find('input',value=re.compile(r'Search|Search this website...|...|Δώστε τίτλο|Αναζήτηση|αναζήτηση|SEARCH ...|Εύρεση|SEARCH|search|ΑΝΑΖΗΤΗΣΗ'))
    search_opt2 = soup.find('input',placeholder=re.compile(r'Search|Search this website...|Αναζήτηση|αναζήτηση|SEARCH ...||SEARCH|search|ΑΝΑΖΗΤΗΣΗ'))
    search_opt3 = soup.find('input',{'id' : re.compile(r'keyword|searchform|acp_magento_search_id_main_page|searchStr|TextBox|txtSearch|[sS]earch|searchstring')})
    search_opt4 = soup.find('input',{'class' : re.compile(r'[S]earch|input|Textbox|assistive-text searchsubmit|searchInput|SEARCH|searchtextfieldlogin|SearchBox|formfieldCopy|search_text|field|go|searchstring')})
    if search_opt1 !=None or search_opt2 !=None or search_opt3 !=None or search_opt4!=None:
        search = 1
    else:
        search = 0
    results.append(search)
    has_blog = soup.find('a',text = re.compile(r'Blog|BLOG|blog'))
    has_blog1 = soup.find('a',href = re.compile(r'Blog|BLOG|blog'))
    if has_blog !=None or has_blog1!=None:
        blog = 1
    else:
        blog = 0
    results.append(blog)
    has_app1 = soup.find('img',src=re.compile(r'google-logo|apple-logo|googleplay|appstore'))
    has_app2 = soup.find('a',href=re.compile(r'apple.com|https://play.google.com'))
    has_app3 = soup.find('a',text=re.compile(r'[aA]pp|APP|[aA]pplication|APPLICATION'))
    if has_app1 !=None or has_app2 !=None or has_app3 !=None:
        mob_app = 1
    else:
        mob_app = 0
    results.append(mob_app)
    results.append(shopping_cart(url))
    return results

In [2]:
def social_networks(sp):
    import re
    import requests
    from bs4 import BeautifulSoup
    
    social_nets = []
    social_urls = []
    fb1 = sp.find_all('img',src=re.compile('[fF]acebook|fb|FACEBOOK'))
    fb2 = sp.find_all('a',href=re.compile(r'[fF]acebook|fb|FACEBOOK'))
    fb3 = sp.find_all(text=re.compile(r'[Ff]acebook|FACEBOOK|fb'))
    if fb1 !=[] or fb2!= [] or fb3!= []:
        if fb1 !=[]:
            social_urls.append(fb1)
        if fb2 !=[]:
            social_urls.append(fb2)
        social_nets.append('Facebook')
    twt1 = sp.find_all('img',src=re.compile(r'[tT]witter|TWITTER'))
    tw2 = sp.find_all('a',href=re.compile(r'[tT]witter|twitter|TWITTER'))
    tw3 = sp.find_all(text=re.compile(r'[tT]witter|TWITTER'))
    if twt1 !=[] or tw2 !=[] or tw3 !=[]:
        if twt1 !=[]:
            social_urls.append(twt1)
        if tw2!=[]:
            social_urls.append(tw2)
        social_nets.append('Twitter')
    linkedin1 = sp.find_all('a',href=re.compile(r'[Ll]inkedin|LINKEDIN'))
    linkedin2 = sp.find_all('a',{'class' :re.compile(r'[Ll]inkedin|LINKEDIN')})
    linkedin3 = sp.find_all('img',src=re.compile(r'[Ll]inkedin|LINKEDIN'))
    if linkedin1 !=[] or linkedin2 !=[] or linkedin3 !=[]:
        if linkedin1 !=[]:
            social_urls.append(linkedin1)
        if linkedin2 !=[]:
            social_urls.append(linkedin2)
        if linkedin3 !=[]:
            social_urls.append(linkedin3)
        social_nets.append('Linkedin')
    insta1 = sp.find_all('a',href= re.compile(r'[iI]nstagram|INSTAGRAM'))
    insta2 = sp.find_all('img',src= re.compile(r'[iI]nstagram|INSTAGRAM'))
    insta3 = sp.find_all('a',{'class' :re.compile(r'[iI]nstagram|INSTAGRAM')})
    if insta1 !=[] or insta2!=[] or insta3!=[]:
        if insta1 !=[]:
            social_urls.append(insta1)
        if insta2 !=[]:
            social_urls.append(insta2)
        if insta3 !=[]:
            social_urls.append(insta3)
        social_nets.append('Instagram')
    yout1 = sp.find_all('a',href= re.compile(r'[yY]outube|YOUTUBE'))
    yout2 = sp.find_all('img',src= re.compile(r'[yY]outube|YOUTUBE'))
    if yout1 !=[] or yout2 !=[]:
        if yout1 !=[]:
            social_urls.append(yout1)
        if yout2 !=[]:
            social_urls.append(yout2)
        social_nets.append('Youtube')
    s_urls = list(set(x for l in social_urls for x in l))
    total_urls = []
    for u in s_urls:
        if u.has_attr('href'):
            total_urls.append(u['href'])
        if u.has_attr('src'):
            total_urls.append(u['src'])
    final = []
    for t in total_urls:
        if '.jpg' not in t and '.png' not in t:
            final.append(t)
    return social_nets,final

In [3]:
def shopping_cart(url):
    import re
    import requests
    from bs4 import BeautifulSoup
    from pprint import pprint
    r = requests.get(url)
    soup10 = BeautifulSoup(r.content,'lxml')
    cart = soup10.find_all('a',href=re.compile(r'[Cc]art|basket|eshop|ESHOP|E-shop|e-shop|E-SHOP|bsk|kalathi|EShop|kala8iagorwn|shop|SHOP|Shop'))
    basket = soup10.find_all('div',{'class' :re.compile(r'[Cc]art|basket|shop|eshop|ESHOP|E-shop|E-SHOP|e-shop|bsk|kalathi|EShop|kala8iagorwn')})
    if cart !=[] or basket !=[]:
        return 1
    else:
        return 0

In [4]:
def check_redirects(url):
    import requests
    
    r = requests.get(url)
    r1 = requests.get(r.url)
    if r.content !=r1.content:
        return r.url,r1
    else:
        return url,r

In [5]:
import pandas as pd
data = pd.read_csv('C:/Users/User/Desktop/diploma thesis Data Science/the_urls.csv', header = None)
urls = list(data[0])
del urls[0]
urls1 = urls[0:2000]
urls2 = urls[2000:len(urls)]

In [ ]:
from requests.exceptions import ConnectionError
total1 = []
c=0
for url in urls1:
    c+=1
    if c%100 == 0:
        print(c)
    try:
        total1.append(first_crawler(url))
    except ConnectionError:
        print(url)
        continue
    except ConnectionResetError:
        print(url)
        continue

In [ ]:
totaldf = pd.DataFrame(total1,columns=['url','social_networks','urls_from_social','multilang_opt','newsletter','search_opt','blog','mob_app','eshop'])
totaldf['social_networks'] = totaldf['social_networks'].apply(lambda x: ','.join(x))
totaldf['urls_from_social'] = totaldf['urls_from_social'].apply(lambda x: ','.join(x))
#totaldf.to_csv('first_crawler_results.csv',encoding='utf-8',index= False)

In [ ]:
from requests.exceptions import ConnectionError
total2 = []
c=0
for url in urls2:
    c+=1
    if c%100 == 0:
        print(c)
    try:
        total2.append(first_crawler(url))
    except ConnectionError:
        print(url)
        continue
    except ConnectionResetError:
        print(url)
        continue

In [ ]:
total = total1+total2
totaldf = pd.DataFrame(total,columns=['url','social_networks','urls_from_social','multilang_opt','newsletter','search_opt','blog','mob_app','eshop'])
totaldf['social_networks'] = totaldf['social_networks'].apply(lambda x: ','.join(x))
totaldf['urls_from_social'] = totaldf['urls_from_social'].apply(lambda x: ','.join(x))
totaldf.to_csv('first_crawler_results.csv',encoding='utf-8',index= False)